# Script to run a Support Vector Machine Model on our transformed data

Done with filtered transaction data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

# Read in our transformed data
loans_base_df = pd.read_csv('./csv_files/loans_merged_filtered_transactions.csv')

# Column Transformer

In [4]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Numerical Variables
num_var = [ #'amount_order','k_symbol_LEASING',
            #'k_symbol_Other', 'k_symbol_POJISTNE',
            #'k_symbol_SIPO', 'k_symbol_UVER',
            'amount_sum',
            'amount_mean', 'amount_std', 
            'balance_min', 'balance_max', 'balance_mean', 'balance_std',
            'type_PRIJEM_sum', 'type_VYBER_sum',
            'type_VYDAJ_sum', 'operation_Other_sum', 'operation_PREVOD NA UCET_sum',
            'operation_PREVOD Z UCTU_sum', 'operation_VKLAD_sum',
            'operation_VYBER_sum', 'operation_VYBER KARTOU_sum',
            #'k_symbol_DUCHOD_sum', 
            'k_symbol_Other_sum', 'k_symbol_POJISTNE_sum',
            'k_symbol_SANKC. UROK_sum', 'k_symbol_SIPO_sum', 'k_symbol_SLUZBY_sum',
            'k_symbol_UROK_sum', 
            #'k_symbol_UVER_sum', 
            'A4', 'A5', 'A6', 'A7', 'A8',
            'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']

# Ordinal Variables
ord_var = ['gender','frequency','type_y']

# Extra Variables
date_var = ['date_x','date_y', 'date_min', 'date_max','birth_date']
drop_var = ['amount_x','duration','payments','loan_id','account_id','district_id','type_x','issued']
#pass_col = 

#Replace inf with nan
loans_base_df = loans_base_df.replace([np.inf, -np.inf], np.nan)

# Set Target Variables
target_raw = loans_base_df['status'].copy()
target_raw[(target_raw == 'A') | (target_raw == 'C')] = 0
target_raw[(target_raw == 'B')  | (target_raw == 'D')] = 1


#Build a column transforrner

# Create our transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = np.nan, strategy='median')),
    ('scaler', StandardScaler())])    
    
binary_transformer = OneHotEncoder(handle_unknown='ignore')

ordinal_transformer = OneHotEncoder(categories  = [ ['M','F'],
                                                     ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'],
                                                     ['No Card','junior','classic','gold']])

# Create the preprocessor
preprocessor = ColumnTransformer(transformers = [ ('numerical', numeric_transformer, num_var),
                                                  ('ordinal',ordinal_transformer, ord_var)],
                                                  #('date','passthrough', date_var)],
                                                   remainder = 'drop')

# Set our X and y variables for use in our model
X = loans_base_df.drop(columns = 'status')
y = target_raw 


#Seperate our train and test splits
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state = 42)


In [5]:
# Use the transformer 

X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.fit_transform(X_val)
y_val = y_val.astype(bool)


In [6]:
# Convert transformed data into dataframe

ord_var_list = ['M','F'] +  ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'] + ['No Card','junior','classic','gold']

transformed_columns = num_var +  ord_var_list # + date_var

print(X_train_transformed.shape)
print(len(transformed_columns))

X_train_transform_df = pd.DataFrame(X_train_transformed, columns = transformed_columns)

X_train_transform_df.head()

(545, 44)
44


,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,M,F,POPLATEK PO OBRATU,POPLATEK TYDNE,POPLATEK MESICNE,No Card,junior,classic,gold
0,0.948068,0.998033,1.086245,-0.115690,0.853546,1.277341,0.399259,0.988251,1.795386,0.697803,0.783667,-0.819272,2.214806,-0.929416,1.124719,1.058555,1.041797,-0.214299,-0.071571,-0.766600,-0.105292,0.783667,-0.340258,-0.543242,1.242068,1.602471,0.270864,-0.151634,-1.376180,-0.726400,-0.895419,-0.792573,-0.500862,-0.435657,-0.420331,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.066088,-1.190396,-1.196385,-0.115690,-1.165943,-0.992643,-1.458940,-0.112171,-0.734089,0.104868,0.119617,1.733027,-0.530665,0.339445,-0.382313,-0.147024,-0.223849,4.601558,-0.071571,0.601190,0.406631,0.119617,-0.405992,-0.454369,-0.033829,0.068361,0.270864,0.916992,0.591623,-0.382954,-0.841526,-0.722240,0.321206,-0.341746,-0.343146,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,-0.489164,-0.252414,0.034337,-0.715090,-0.162213,-0.952224,0.778157,-0.512892,2.014846,-0.860702,-0.391730,-0.819272,-0.530665,-0.038470,-0.320253,-0.147024,-0.433401,-0.214299,0.697025,-0.766600,-0.470951,-0.391730,0.167111,-1.194980,-1.373521,-1.246589,-0.629263,-1.576469,1.593049,0.909387,0.990845,0.886040,-0.890262,0.172850,0.083187,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.867363,0.699664,0.823531,-0.013229,0.598120,-0.339217,0.072504,0.820330,1.115708,0.823854,-0.094028,2.142500,1.997280,-0.901308,0.523921,-0.147024,0.661869,-0.214299,-0.071571,2.815221,4.526394,-0.094028,-0.587681,-0.543242,-0.352803,0.287520,-0.629263,-0.151634,-0.815382,-0.341464,0.128553,0.023289,0.148139,-0.468767,-0.453821,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.702571,-0.844159,-0.883131,0.089233,-0.943938,-0.543851,-1.215030,-0.710423,-0.734089,-0.644221,-0.527622,-0.086150,-0.530665,-0.225357,-0.717954,-0.147024,-0.676052,-0.214299,-0.071571,-0.484337,-0.324688,-0.527622,2.675973,-1.194980,-1.373521,-1.246589,-0.629263,-1.576469,1.593049,2.344639,-1.434351,-1.463080,2.008608,2.707004,2.716704,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [7]:
# Check for nan's

X_train_transform_df.isna().sum()

amount_sum                      0
amount_mean                     0
amount_std                      0
balance_min                     0
balance_max                     0
balance_mean                    0
balance_std                     0
type_PRIJEM_sum                 0
type_VYBER_sum                  0
type_VYDAJ_sum                  0
operation_Other_sum             0
operation_PREVOD NA UCET_sum    0
operation_PREVOD Z UCTU_sum     0
operation_VKLAD_sum             0
operation_VYBER_sum             0
operation_VYBER KARTOU_sum      0
k_symbol_Other_sum              0
k_symbol_POJISTNE_sum           0
k_symbol_SANKC. UROK_sum        0
k_symbol_SIPO_sum               0
k_symbol_SLUZBY_sum             0
k_symbol_UROK_sum               0
A4                              0
A5                              0
A6                              0
A7                              0
A8                              0
A9                              0
A10                             0
A11           

# Build and Run The Model

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import eli5

#Initialize Model
sv_model = SVC(random_state = 42)

sv_model.fit(X_train_transform_df, y_train.astype(bool))


/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [9]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Initialize our tuning grid
param_grid = {    
    'C' : np.logspace(0,3, num = 4), 
    'kernel' : ['linear','poly','rbf'],
    'gamma' : np.logspace(-1,2, num = 4)
}

#Set the grid search
grid_search = RandomizedSearchCV(sv_model, param_grid, cv=5, verbose=2, random_state=42, n_iter=300, scoring='balanced_accuracy')

#Fit the model
grid_search.fit(X_train_transform_df, y_train.astype(bool))

print(("best RF from grid search: %.3f"
       % grid_search.score(X_val_transformed, y_val)))

/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 48 is smaller than n_iter=300. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] kernel=linear, gamma=0.1, C=1.0 .................................
[CV] .................. kernel=linear, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=1.0 .................................
[CV] .................. kernel=linear, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=1.0 .................................
[CV] .................. kernel=linear, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=1.0 .................................
[CV] .................. kernel=linear, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=1.0 .................................
[CV] .................. kernel=linear, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=1.0 ...................................
[CV] .................... kernel=poly, gamma=0.1, C=1.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=1.0 ...................................
[CV] ..........

[CV] ................. kernel=linear, gamma=0.1, C=10.0, total=   0.1s
[CV] kernel=linear, gamma=0.1, C=10.0 ................................
[CV] ................. kernel=linear, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=10.0 ................................
[CV] ................. kernel=linear, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=linear, gamma=0.1, C=10.0 ................................
[CV] ................. kernel=linear, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=10.0 ..................................
[CV] ................... kernel=poly, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=10.0 ..................................
[CV] ................... kernel=poly, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=10.0 ..................................
[CV] ................... kernel=poly, gamma=0.1, C=10.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=10.0 ..................................
[CV] .

[CV] ................ kernel=linear, gamma=0.1, C=100.0, total=   0.4s
[CV] kernel=linear, gamma=0.1, C=100.0 ...............................
[CV] ................ kernel=linear, gamma=0.1, C=100.0, total=   0.2s
[CV] kernel=linear, gamma=0.1, C=100.0 ...............................
[CV] ................ kernel=linear, gamma=0.1, C=100.0, total=   0.3s
[CV] kernel=linear, gamma=0.1, C=100.0 ...............................
[CV] ................ kernel=linear, gamma=0.1, C=100.0, total=   1.1s
[CV] kernel=linear, gamma=0.1, C=100.0 ...............................
[CV] ................ kernel=linear, gamma=0.1, C=100.0, total=   1.3s
[CV] kernel=poly, gamma=0.1, C=100.0 .................................
[CV] .................. kernel=poly, gamma=0.1, C=100.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=100.0 .................................
[CV] .................. kernel=poly, gamma=0.1, C=100.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=100.0 .................................
[CV] .

[CV] ............... kernel=linear, gamma=0.1, C=1000.0, total=   3.5s
[CV] kernel=linear, gamma=0.1, C=1000.0 ..............................
[CV] ............... kernel=linear, gamma=0.1, C=1000.0, total=   3.4s
[CV] kernel=linear, gamma=0.1, C=1000.0 ..............................
[CV] ............... kernel=linear, gamma=0.1, C=1000.0, total=   6.1s
[CV] kernel=linear, gamma=0.1, C=1000.0 ..............................
[CV] ............... kernel=linear, gamma=0.1, C=1000.0, total=   5.2s
[CV] kernel=linear, gamma=0.1, C=1000.0 ..............................
[CV] ............... kernel=linear, gamma=0.1, C=1000.0, total=   3.3s
[CV] kernel=poly, gamma=0.1, C=1000.0 ................................
[CV] ................. kernel=poly, gamma=0.1, C=1000.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=1000.0 ................................
[CV] ................. kernel=poly, gamma=0.1, C=1000.0, total=   0.0s
[CV] kernel=poly, gamma=0.1, C=1000.0 ................................
[CV] .

best RF from grid search: 0.746


[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:  2.0min finished


In [10]:
# Put our results in a dataframe

results  = pd.DataFrame(grid_search.cv_results_)

y_val.sum()

16

In [11]:
# See the best scores
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009654,0.002623,0.001540,0.000244,linear,0.1,1,"{'kernel': 'linear', 'gamma': 0.1, 'C': 1.0}",0.708333,0.734536,0.625000,0.687715,0.656357,0.682388,0.038450,1
3,0.009004,0.001785,0.001415,0.000114,linear,1,1,"{'kernel': 'linear', 'gamma': 1.0, 'C': 1.0}",0.708333,0.734536,0.625000,0.687715,0.656357,0.682388,0.038450,1
9,0.008908,0.001664,0.001410,0.000265,linear,100,1,"{'kernel': 'linear', 'gamma': 100.0, 'C': 1.0}",0.708333,0.734536,0.625000,0.687715,0.656357,0.682388,0.038450,1
6,0.008879,0.001705,0.001351,0.000144,linear,10,1,"{'kernel': 'linear', 'gamma': 10.0, 'C': 1.0}",0.708333,0.734536,0.625000,0.687715,0.656357,0.682388,0.038450,1
39,4.821123,1.135721,0.001759,0.000435,linear,1,1000,"{'kernel': 'linear', 'gamma': 1.0, 'C': 1000.0}",0.692869,0.729381,0.625000,0.682560,0.656357,0.677234,0.035090,5
42,5.819324,1.418817,0.001782,0.000020,linear,10,1000,"{'kernel': 'linear', 'gamma': 10.0, 'C': 1000.0}",0.692869,0.729381,0.625000,0.682560,0.656357,0.677234,0.035090,5
36,4.315940,1.149005,0.001447,0.000094,linear,0.1,1000,"{'kernel': 'linear', 'gamma': 0.1, 'C': 1000.0}",0.692869,0.729381,0.625000,0.682560,0.656357,0.677234,0.035090,5
45,5.929750,1.572993,0.001857,0.000089,linear,100,1000,"{'kernel': 'linear', 'gamma': 100.0, 'C': 1000.0}",0.692869,0.729381,0.625000,0.682560,0.656357,0.677234,0.035090,5
33,0.706849,0.477607,0.001716,0.000365,linear,100,100,"{'kernel': 'linear', 'gamma': 100.0, 'C': 100.0}",0.692869,0.698024,0.625000,0.682560,0.656357,0.670962,0.027102,9
30,0.647085,0.438649,0.001593,0.000301,linear,10,100,"{'kernel': 'linear', 'gamma': 10.0, 'C': 100.0}",0.692869,0.698024,0.625000,0.682560,0.656357,0.670962,0.027102,9


# Confusion Matrix for our best results

In [12]:
y_pred_val = grid_search.predict(X_val_transformed)

from sklearn.metrics import confusion_matrix

x = confusion_matrix(y_val, y_pred_val)

tn = x[0,0]  
fp = x[0,1] 
fn = x[1,0] 
tp = x[1,1]

print([tp,fp])
print([fn,tn])

[8, 1]
[8, 120]


In [13]:
# Feature Importances

eli5.show_weights(grid_search.best_estimator_, feature_names = transformed_columns)

In [14]:
# See out best parameters

grid_search.best_params_

{'kernel': 'linear', 'gamma': 0.1, 'C': 1.0}

# Apply Model To Remaining Customers

In [15]:
# Read in our data that doesn't have loan info
account_df = pd.read_csv("./csv_files/accounts_merged_all_transactions.csv")

# Run the transformer on it
account_transformed = preprocessor.fit_transform(account_df)

# Put it into a Dataframe
account_transform_df = pd.DataFrame(account_transformed, columns = transformed_columns)

In [16]:
account_transform_df['account_id'] = account_df['account_id']

In [17]:
account_transform_df.shape

(4500, 45)

In [18]:
loans_base_df.shape

(682, 58)

In [19]:
# Find how many accounts don't already have loans...
(~account_transform_df['account_id'].isin(loans_base_df['account_id'])).sum()

3818

In [20]:
# Check for Na's
account_df.isna().sum()

account_id                         0
district_id                        0
frequency                          0
date                               0
amount_order                     742
k_symbol_LEASING                 742
k_symbol_Other                   742
k_symbol_POJISTNE                742
k_symbol_SIPO                    742
k_symbol_UVER                    742
date_min                           0
date_max                           0
amount_sum                         0
amount_mean                        0
amount_std                         0
balance_min                        0
balance_max                        0
balance_mean                       0
balance_std                        0
type_PRIJEM_sum                    0
type_VYBER_sum                     0
type_VYDAJ_sum                     0
operation_Other_sum                0
operation_PREVOD NA UCET_sum       0
operation_PREVOD Z UCTU_sum        0
operation_VKLAD_sum                0
operation_VYBER_sum                0
o

In [21]:
# Filter our dataframe to remove all clients that already have loans

account_no_loan_df = account_transform_df[(~account_transform_df['account_id'].isin(loans_base_df['account_id']))]

account_no_loan_df = account_no_loan_df.drop(columns='account_id')

account_no_loan_df.head()

,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,M,F,POPLATEK PO OBRATU,POPLATEK TYDNE,POPLATEK MESICNE,No Card,junior,classic,gold
0,-0.764355,-1.039177,-1.078609,0.241096,-1.080353,-1.363367,-1.175905,-0.780242,-0.446157,-0.763416,-0.711272,-0.324241,-0.017876,-0.791302,-0.733753,-0.3084,-0.733180,-0.177685,-0.085004,-0.094362,-0.016410,-0.711272,-0.554164,0.596904,-0.520953,-0.806169,-0.667309,-0.526598,-0.199264,-0.412591,-0.012606,-0.068990,0.426891,-0.479531,-0.460300,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,-0.824695,-0.803010,-0.824016,0.241096,-0.498533,-0.639317,-0.311688,-0.811965,-0.446157,-0.864195,-0.943883,-0.532854,-0.382912,-0.566646,-0.777120,-0.3084,-0.765341,1.343970,-0.085004,-0.713509,-0.695430,-0.943883,-0.484618,0.742767,0.595782,-0.350477,-0.667309,0.160169,-0.902388,-0.156985,0.530235,0.438841,-0.134555,-0.446989,-0.424173,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,-0.782576,-0.960475,-1.031684,0.156333,-0.967813,-0.983555,-1.005238,-0.783186,-0.446157,-0.801456,-0.786157,-0.319623,0.033382,-0.829804,-0.772303,-0.3084,-0.868751,-0.177685,-0.085004,-0.375369,-0.293048,-0.786157,-0.450415,1.297046,0.530092,0.105214,-0.667309,0.160169,-0.568531,-0.573946,0.530235,0.382416,0.686020,-0.402856,-0.397701,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,-0.921149,-0.935717,-1.026603,0.071571,-1.039404,-0.763526,-1.037719,-0.924732,-0.446157,-0.954134,-1.062528,-0.707361,-0.172732,-0.830113,-0.818927,-0.3084,-0.920199,-0.177685,-0.085004,-0.545602,-0.670281,-1.062528,-0.587387,-0.511655,-0.323882,0.333060,-0.667309,-0.183214,-0.877096,-0.354533,0.150246,0.048564,0.124574,-0.475630,-0.461291,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5,-0.559317,-0.775510,-0.885890,0.198715,-0.538143,-0.139854,-0.712401,-0.551504,-0.446157,-0.559644,0.114799,0.229405,0.367025,-0.829649,-0.681145,-0.3084,-0.856645,-0.177685,-0.085004,0.557725,0.134483,0.114799,-0.411124,-0.074066,0.464401,0.333060,1.180055,1.877088,0.063775,-0.734548,-0.175458,-0.247671,0.426891,-0.414298,-0.398628,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


# Predict on customers without loans

In [22]:
# Predict thier loan status using our best estimator from above

y_acc_pred_val = grid_search.best_estimator_.predict(account_no_loan_df)

In [23]:
no_loans_df = account_no_loan_df[y_acc_pred_val==1]

preapp_loans_df = account_no_loan_df[y_acc_pred_val==0]

# See how many are predicted of each
print(no_loans_df.shape)
print(preapp_loans_df.shape)

(166, 44)
(3652, 44)


In [24]:
# Check on the differences between the two groups

X_zero = no_loans_df
X_one = preapp_loans_df

A = (X_one.describe().T - X_zero.describe().T)

print(A.sort_values(by ='mean', ascending=False))

                               count      mean       std        min       25%  \
balance_min                   3486.0  2.685645 -2.519850  16.160429  3.193646   
operation_PREVOD NA UCET_sum  3486.0  0.816237  0.626618   0.000000  0.219869   
operation_VYBER KARTOU_sum    3486.0  0.247232  0.650381   0.000000  0.000000   
k_symbol_SIPO_sum             3486.0  0.214373 -0.411483   0.000000  0.032416   
POPLATEK MESICNE              3486.0  0.125137 -0.163790   0.000000  0.000000   
A9                            3486.0  0.095907  0.023609   0.000000  0.000000   
operation_PREVOD Z UCTU_sum   3486.0  0.095649  0.172885   0.000000  0.000000   
classic                       3486.0  0.081051  0.109718   0.000000  0.000000   
A8                            3486.0  0.076889 -0.022381   0.000000  0.000000   
k_symbol_POJISTNE_sum         3486.0  0.072449  0.401386   0.000000  0.000000   
A13                           3486.0  0.068216 -0.004725   0.000000  0.105798   
balance_mean                